In [2]:
import numpy as np
import pandas as pd
import opensmile


import pydub
import librosa
from scipy import signal
from mutagen.mp3 import MP3
import numpy as np
from numpy import sign, trapz
from teager_py import Teager
import statsmodels.api as sm
import wave

import warnings
warnings.filterwarnings('ignore')

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [61]:
import imp

print(imp.find_module('opensmile'))


(None, '/home/xuranzeng/.local/lib/python3.8/site-packages/opensmile', ('', '', 5))


In [3]:
# convert librosa to AudioSegment
def librosa_to_audiosegment(filename):
    y, sr = librosa.load(filename)
    # convert from float to uint16
    y = np.array(y * (1<<15), dtype=np.int16)
    audio_segment = pydub.AudioSegment(
        y.tobytes(), 
        frame_rate=sr,
        sample_width=y.dtype.itemsize, 
        channels=1
    )
    return audio_segment

In [1]:
config_str = '''
;;;;;;; component list ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

[componentInstances:cComponentManager]
instance[dataMemory].type=cDataMemory
printLevelStats=0

[framer:cFramer]
reader.dmLevel = wave
writer.dmLevel = frames
copyInputName = 1
frameMode = fixed
frameSize = 0.025000
frameStep = 0.010000
frameCenterSpecial = left
noPostEOIprocessing = 1
;;;;;;;;;;;;;;;;;;;;;;;;;;;; main section ;;;;;;;;;;;;;;;;;;;;;;;;;;;

\{\cm[source{?}:include external source]}
\{GeMAPSv01b_core.lld.conf.inc}
\{GeMAPSv01b_core.func.conf.inc}

;;;;;;;; prepare for standard data output ;;;;;;;;;;;;;;;;;;;;;

[componentInstances:cComponentManager]
instance[lldconcat].type=cVectorConcat
instance[funcconcat].type=cVectorConcat

[lldconcat:cVectorConcat]
reader.dmLevel = gemapsv01b_lldsetE_smo;gemapsv01b_lldsetF_smo
writer.dmLevel = lld
includeSingleElementFields = 1

[funcconcat:cVectorConcat]
reader.dmLevel = gemapsv01b_functionalsF0;gemapsv01b_functionalsLoudness;gemapsv01b_functionalsMeanStddevVoiced;gemapsv01b_functionalsMeanSpectralUnvoiced;gemapsv01b_temporalSet
writer.dmLevel = func
includeSingleElementFields = 1

\{\cm[sink{?}:include external sink]}

'''

In [9]:
with open('my.conf', 'w') as fp:
    fp.write(config_str)

smile = opensmile.Smile(
    feature_set='my.conf',
    feature_level='func',
)
print(smile.feature_names)

OpenSmileException: Code: 6

In [77]:
with open('my.conf', 'w') as fp:
    fp.write(config_str)

In [20]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    #feature_set='my.conf',
    
    #feature_level=opensmile.FeatureLevel.Functionals,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)
#print(smile.feature_names)

In [28]:
def get_GeMAPS1(filepath):
    audio = librosa_to_audiosegment(filepath) # convert librosa array to audiosegment
    sr = audio.frame_rate # get sample rate
    audio = np.array(audio.get_array_of_samples()) # convert audiosegment to array
    df = smile.process_signal(audio,sr).to_numpy() # get GeMAPS
    return df

def get_GeMAPS(path):
    GeMAPS = []
    for i in range(0,len(path)):
        filepath = path.loc[i,'Path']
        audio = librosa_to_audiosegment(filepath) # convert librosa array to audiosegment
        sr = audio.frame_rate # get sample rate
        audio = np.array(audio.get_array_of_samples()) # convert audiosegment to array
        df = smile.process_signal(audio,sr) # get GeMAPS
        GeMAPS.append(pd.DataFrame(df.mean(axis=0)).T) # take average
        print('processing:',i)
    
    GeMAPS = pd.concat(GeMAPS).reset_index()
    return GeMAPS

In [15]:
# load path
path = pd.read_csv('/project/graziul/ra/team_ser/susas_path.csv')
path.drop(path.iloc[:, 0:1], inplace = True, axis = 1)
path = path[['Path','Arousal','Valence']]

filepath = path.loc[0,'Path']
filepath

'/project/graziul/data/corpora/susas/speech/actual/roller/f1/free_oov_all/all1.sph'

In [29]:
a=get_GeMAPS1 (filepath)

In [30]:
a

array([[   6.2094   ,   -6.692044 ,   18.515762 , ..., 2372.8406   ,
         577.22797  , -201.       ],
       [   6.094591 ,   -7.1967063,   19.163605 , ..., 2425.487    ,
         807.9421   , -201.       ],
       [   6.053633 ,   -7.5610566,   20.287848 , ..., 2423.443    ,
         784.4694   , -201.       ],
       ...,
       [   6.3575745,   -6.199922 ,   16.613966 , ..., 2480.6326   ,
         630.1203   , -201.       ],
       [   6.4271927,   -6.3704796,   16.449568 , ..., 2492.7798   ,
         547.5786   , -201.       ],
       [   6.3886313,   -6.778122 ,   17.756926 , ..., 2478.921    ,
         698.3128   , -201.       ]], dtype=float32)

In [34]:
a.std(axis=0)

array([1.0112112e-01, 4.5510450e-01, 7.8987169e-01, 9.3159815e-03,
       4.1772798e-03, 2.5513726e-01, 1.0618246e+00, 1.6395235e+00,
       2.2036190e+00, 2.6156530e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       3.4938229e+01, 1.4659906e+02, 0.0000000e+00, 7.0289047e+01,
       1.8171678e+02, 0.0000000e+00, 6.5438782e+01, 1.5797600e+02,
       0.0000000e+00], dtype=float32)

In [35]:
#GeMAPS_0105 = get_GeMAPS(path)

In [10]:
GeMAPS_0105_features = pd.concat([path, GeMAPS_0105.iloc[:,1:]], axis=1)
GeMAPS_0105_features.head(2)

,Path,Arousal,Valence,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,F0semitoneFrom27.5Hz_sma3nz,jitterLocal_sma3nz,...,HNRdBACF_sma3nz,logRelF0-H1-H2_sma3nz,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3amplitudeLogRelF0_sma3nz
0,/project/graziul/data/corpora/susas/speech/act...,High,Negative,6.292282,-6.763290,18.529961,-0.086114,-0.002062,0.0,0.0,...,0.0,0.0,0.0,619.362793,730.045105,-201.0,1487.981445,-201.0,2418.518311,-201.0
1,/project/graziul/data/corpora/susas/speech/act...,High,Negative,6.322670,-6.638078,18.540451,-0.087078,-0.001739,0.0,0.0,...,0.0,0.0,0.0,627.472534,710.429443,-201.0,1501.422119,-201.0,2410.086670,-201.0


In [16]:
#GeMAPS_0105 = get_GeMAPS(path)

In [14]:
#GeMAPS_0105_features.to_csv('/project/graziul/ra/team_ser/GeMAPS_0105.csv')